# Double bonding curve liquidity pool as automatic swap market maker

In [59]:
import chart_studio.plotly as py
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import requests
import ssl
import time
import math
import random
import decimal
import plotly.express as px


#disable ssl for cryptory API & virtualenv
ssl._create_default_https_context = ssl._create_unverified_context
np.set_printoptions(suppress=True)

Request Data from Compound.finance API. We request a time step every hour over the last 9 months. The API does not let you get more granular data per request so we do it in 3 loops.

In [111]:
secondsInMonth = 60 * 60 * 24 *30
cDaiAddress = '0xf5dce57282a584d2746faf1593d3121fcac444dc'
endTimestamp = math.floor(time.time())
    
startTimeStamp = endTimestamp - 3 * 30 * 24 * 60 * 60 #use the most recent 3 month's worth of data.
num_buckets = 3 * 24 * 30 #interest rate every hour

interestOverTime = pd.DataFrame()

for i in range(2,-1,-1): # we can pull up to 3 months of data at a time if taking data points every hour.
    endTimestamp = math.floor(time.time()) - i * secondsInMonth * 3
    startTimeStamp = endTimestamp - 3 * 30 * 24 * 60 * 60 #use three months before the start
    print("itteration %s start: %s. end: %s. num_buckets: %s" %(i, startTimeStamp, endTimestamp, num_buckets))
    requestURL = "https://api.compound.finance/api/v2/market_history/graph?asset=%s&min_block_timestamp=%s&max_block_timestamp=%s&num_buckets=%s"% (cDaiAddress, startTimeStamp, endTimestamp, num_buckets)
    response = requests.get(requestURL)
#     print(response.json())
    outputFrame = pd.DataFrame.from_dict(response.json()['supply_rates'])
    interestOverTime = interestOverTime.append(outputFrame, ignore_index = True)
interestOverTime.head()

itteration 2 start: 1549991968. end: 1557767968. num_buckets: 2160
itteration 1 start: 1557767969. end: 1565543969. num_buckets: 2160
itteration 0 start: 1565543971. end: 1573319971. num_buckets: 2160


,block_number,block_timestamp,rate
0,7710751,1555427968,0.000000
1,7711711,1555442368,0.000000
2,7715071,1555492768,0.001255
3,7716031,1555507168,0.002507
4,7717951,1555535968,0.002499


Convert Data into dataframe

In [114]:
interestOverTime['block_time'] = pd.to_datetime(interestOverTime['block_timestamp'],unit='s')
interestOverTime['rate_per_block'] = interestOverTime['rate'] / (4 * 60 * 24 * 365)
interestOverTime['rate_per_hour'] = interestOverTime['rate'] / (24 * 365)
interestOverTime.tail()
interestOverTime = interestOverTime.drop(interestOverTime.head(100).index) # drop last n rows becuase the rate was much lower here
interestOverTime = interestOverTime.reset_index(drop = True)

In [4]:
len(interestOverTime)

4534

In [5]:
# d1 = pd.DataFrame(0.1/(24*365), index=np.arange(len(interestOverTime)), columns=np.arange(1))
# interestOverTime['rate_per_hour'] = d1

# d2 = pd.DataFrame(0.1, index=np.arange(len(interestOverTime)), columns=np.arange(1))
# interestOverTime['rate'] = d2

Plot Intrest rate over time

In [115]:
data = [go.Scatter(
        x=interestOverTime['block_time'],
        y=interestOverTime['rate']
    )]

layout = go.Layout(
    title='Compound Interst Rate Over time',
    yaxis=dict(title='Lending Rate (%)'),
    xaxis=dict(title='Date'),
    template='plotly_white')

figure = go.Figure(data=data, layout=layout)
figure.show()

Define some helper functions.

In [108]:
#Defines the fixed rate offered by the pool as a function of utilization. See Notion for explinaiton
def poolFixedRate(poolUtilization, currentFloatingRate, isLong):
    alpha = 1.5
    beta = 0
    if isLong:
        return currentFloatingRate * (1 + poolUtilization/alpha + beta)
    if not isLong:
        return currentFloatingRate * (1 - poolUtilization/alpha - beta)

In [8]:
poolFixedRate(0.75,.16,False)

0.084

In [9]:
#Future value NACC(nominal anual, compounding continiously). Assumes t1 and t2 are in seconds
def futureValueNACC(nominal, rate, t1, t2):
    return nominal * math.exp(rate * (t2 - t1) / (60 * 60 * 24 *365))

In [10]:
#Future value NACH(nominal anual, compounding hourly) Assumes t1 and t2 are in seconds
def futureValueNACH(nominal, rate, t1, t2):
    return nominal * (1 + rate/(24*365)) ** (((t2 - t1) * 24 * 365) / (60 * 60 * 24 * 365))

In [11]:
int(interestOverTime.head(1)['block_timestamp'])

1556831679

In [12]:
int(interestOverTime.tail(1)['block_timestamp'])

1573312483

In [13]:
#Finds the index in the pandas frame that has the closest starting index to the time specified
def findClosestIndex(t):
    for i in range(0,len(interestOverTime['block_timestamp'])):
        if t >= interestOverTime.loc[i,'block_timestamp'] and t <= interestOverTime.loc[i + 1,'block_timestamp']:
            return(i)
    return(-1)

In [14]:
#Floating Value accrued between two time periods in the compound data set.
def floatingValue(nominal, t1, t2):
    assert t1 < t2, "invalid input time"
    assert t1 >= int(interestOverTime.head(1)['block_timestamp']), "t1 before time series start"
    assert t2 <= int(interestOverTime.tail(1)['block_timestamp']), "t2 after time series end"
    startingIndex = findClosestIndex(t1)
    endingIndex = findClosestIndex(t2)
    positionValue = nominal
    for i in range(startingIndex, endingIndex + 2):
        ratePerHour = interestOverTime.loc[i,'rate']/(24 * 365)
        positionValue = positionValue * (1 + ratePerHour)
    return positionValue

In [78]:
#calculate the effective APR for a position. v1 is value at time t1. v2 is value at time t2.
#algebraically this comes from taking v2=v1(1+r/(24*365))^((t2-t1)/(60*60)) and solving for r. Then * 100
def effectiveAPR(v1,v2, t1, t2):
    return -8760 * (v2/v1) ** (-3600/(t1 - t2)) * ((v2/v1) ** (3600/(t1 - t2)) - 1)

In [16]:
# when someone goes short how much reserve do we need to keep for them? 
# this defines the lequidity utilization we needed to be reserved for the position.
# not that this equation is not optimal as we over-alocate for the short position by a small amount.
# this would be correct if it considered the change in pool utilization and how this inflences the 
# rate. becuase it doesent consider this it slightly over alocates reserve, which is fine.
def reserveRequirementShort(nominal, rate, poolUtilization, poolTotal, t1, t2):
    newPoolUtilization = (poolUtilization + nominal) / poolTotal
    offeredRate = poolFixedRate(newPoolUtilization, rate, False)
    print("offeredRate", offeredRate)
    return futureValueNACH(nominal, offeredRate, t1, t2) - nominal

In [17]:
reserveRequirementShort(100, 0.05, 0, 1000, 0, 60*60*24*365)

offeredRate 0.0425


4.341594880072108

In [18]:
# when someone goes long how much reserve do we need to keep for them? 
# this defines the lequidity utilization we needed to be reserved for the position
# note that with long we dont know what the potentual upside of the position is
# becuase there is no defined upper bound on this. For the case of this implementation
# we will assume that intrest rates will not exceed 30% and so will reserve future value
# grown at 30% for the position to ensure that there is always lequidity. 
def reserveRequirementLong(nominal, rate, poolUtilization, poolTotal, t1, t2):
    return futureValueNACH(nominal, 0.3, t1, t2) - nominal

Next, simulate the pool value over time with different positions taken out against it. To begin with let's do a simple calculation where we put some arbitary positions against the pools to make sure things make sense. After this we can simulate different positions being taken against the pool.

In [19]:
interestOverTime

,block_number,block_timestamp,rate,block_time,rate_per_block,rate_per_hour
0,7804310,1556831679,0.026341,2019-05-02 21:14:39,1.252912e-08,0.000003
1,7805750,1556853279,0.023087,2019-05-03 03:14:39,1.098135e-08,0.000003
2,7805990,1556856879,0.077711,2019-05-03 04:14:39,3.696321e-08,0.000009
3,7806950,1556871279,0.116131,2019-05-03 08:14:39,5.523736e-08,0.000013
4,7807190,1556874879,0.076435,2019-05-03 09:14:39,3.635592e-08,0.000009
5,7808630,1556896479,0.062027,2019-05-03 15:14:39,2.950298e-08,0.000007
6,7810550,1556925279,0.057888,2019-05-03 23:14:39,2.753417e-08,0.000007
7,7811270,1556936079,0.057745,2019-05-04 02:14:39,2.746621e-08,0.000007
8,7811750,1556943279,0.057565,2019-05-04 04:14:39,2.738065e-08,0.000007
9,7814630,1556986479,0.057496,2019-05-04 16:14:39,2.734801e-08,0.000007


In [20]:
# Say the pool has 1000 dai in it
N = 1000
t1 = int(interestOverTime.head(1)['block_timestamp'])
t2 = int(interestOverTime.tail(1)['block_timestamp'])
startingRate = float(interestOverTime.head(1)['rate'])

#find the future value of the market position
moneyMarketFutureValue = floatingValue(N,t1,t2)

#find the effective marketAPR
effectiveMarketApr = effectiveAPR(N,moneyMarketFutureValue,t1,t2)

print("If the pool had just invested into the money market it would be worth: %s with APR: %s" % 
      (moneyMarketFutureValue, effectiveMarketApr))
p_long = p_short = N/2 #long and short pools are half of the nominal

If the pool had just invested into the money market it would be worth: 1049.977582712235 with APR: 9.331956831555233


In [21]:
# say someone goes long with 500 dai and someone goes short with 500 dai. 
# both positions are opened at the same time at the start of the data set
# This will result in half pool utilization for both trades
# for this part of the simulation assume that they hold the full value to maturity.

swap_N_short = 500
swap_N_long = 500

longFixedRate = poolFixedRate(swap_N_long/N,startingRate,True) #Fixed rate counterparty will pay
shortFixedRate = poolFixedRate(swap_N_short/N,startingRate,False) #Fixed rate counterparty will recive
print("Floating Rate: %s\nLong offer(trader long)   -> pool recives fixed rate: %s & pays float\nShort offer(trader short) -> pool pays fixed rate:    %s & recives floating rate" %
      (startingRate, longFixedRate, shortFixedRate))

Floating Rate: 0.02634122012667002
Long offer(trader long)   -> pool recives fixed rate: 0.03556064717100453 & pays float
Short offer(trader short) -> pool pays fixed rate:    0.017121793082335513 & recives floating rate


In [22]:
# now find the future value of the position, from the perspective of the lequidity pool.
# As a result we are paying the lower fixed rate and reciving the higher fixed rate.

# in the long offer swap the lequidity pool is short: reciving a fixed rate and paying a floating rate
SwapNPV_long = futureValueNACH(swap_N_long, longFixedRate, t1, t2) - floatingValue(swap_N_long, t1,t2)
print(SwapNPV_long)

# in the short offer swap the lequidity pool is long: reciving floating rate, paying fixed rate
SwapNPV_short = floatingValue(swap_N_short, t1,t2) - futureValueNACH(swap_N_short, shortFixedRate, t1, t2)
print(SwapNPV_short)

FinalPoolValue = SwapNPV_short + SwapNPV_long + moneyMarketFutureValue
print("Final value of pool after offering swaps is %s with an APR of %s. Was pool profitable?: %s" %
      (FinalPoolValue, effectiveAPR(N,FinalPoolValue,t1,t2), FinalPoolValue >= moneyMarketFutureValue))

-15.609882980749035
20.494770639702608
Final value of pool after offering swaps is 1054.8624703711887 with an APR of 10.220133407976958. Was pool profitable?: True


## Simulate hyper parameters over provided functions

Next we will sweep over long and short pool sizes against different positions in the data set for different swap maturities. Then run this over a ranger of different alpha values.

In [70]:
# simulate swap over duration
def swapSimulate(swap_N_long, swap_N_short,startingRate, t1, t2, totalPool):
    # find long and short offered rates
    longFixedRate = poolFixedRate(swap_N_long/N,startingRate,True) # Fixed rate counterparty will pay
    shortFixedRate = poolFixedRate(swap_N_short/N,startingRate,False) # Fixed rate counterparty will recive

    # find value of swaps from lequidity pools perspective
    SwapNPV_long = futureValueNACH(swap_N_long, longFixedRate, t1, t2) - floatingValue(swap_N_long, t1,t2)
    SwapNPV_short = floatingValue(swap_N_short, t1,t2) - futureValueNACH(swap_N_short, shortFixedRate, t1, t2)

    # find growth of underlying during duration
    moneyMarketFutureValue = floatingValue(N,t1,t2)

    # balance the sides and the underlying at maturity
    FinalPoolValue = SwapNPV_short + SwapNPV_long + moneyMarketFutureValue
    effectiveAPRValue = effectiveAPR(N,FinalPoolValue,t1,t2)
    return(effectiveAPRValue)

In [116]:
results = []
poolReserve = N/2

for i in range(0,5):
    # Add the new dataframe to store new results
    results.append(pd.DataFrame())
    for short_pool in range(0,600,100):
        for long_pool in range(0,600,100):
            # re-calculate t1 and t2 based on the new data set.
            # t1 is found by shifting the initial starting time stamp forward 1/5 a fraction of the
            # total time every loop. This gives us a equal slice over the whole time series data
            t1 = int(int(interestOverTime.head(1)['block_timestamp']) + (int(interestOverTime.tail(1)['block_timestamp']) - int(interestOverTime.head(1)['block_timestamp']))/5 * i)
            t2 = int(interestOverTime.tail(1)['block_timestamp'])
            
            # find the starting rate at this spesific point in time
            startingRate = float(interestOverTime.loc[int(len(interestOverTime)/5 * i),'rate'])
            apr = swapSimulate(long_pool, short_pool ,startingRate ,t1 ,t2, poolReserve)
            
            # find the effective market APR to compare to the pool
            moneyMarketFutureValue = floatingValue(N,t1,t2)
            effectiveMarketApr = effectiveAPR(N,moneyMarketFutureValue,t1,t2)

            profit = (apr - effectiveMarketApr) / apr
            print("i: %s, p_short: %s, p_long: %s, r: %s, apr: %s, effectiveApr: %s, profit: %s" 
                  %(i, short_pool, long_pool, round(startingRate,5), round(apr,5), round(effectiveMarketApr,5), round(profit,5)))
            newResult = pd.DataFrame({"longPool": long_pool/poolReserve,
                                      "shortPool": short_pool/poolReserve,
                                      "apr": apr,
                                      "profit": profit,
                                      "profitable": int(profit>0)}, index=[0])
            results[i] = results[i].append(newResult, ignore_index = True)
            

i: 0, p_short: 0, p_long: 0, r: 0.10818, apr: 0.09352, effectiveApr: 0.09352, profit: 0.0
i: 0, p_short: 0, p_long: 100, r: 0.10818, apr: 0.09572, effectiveApr: 0.09352, profit: 0.02297
i: 0, p_short: 0, p_long: 200, r: 0.10818, apr: 0.09937, effectiveApr: 0.09352, profit: 0.05889
i: 0, p_short: 0, p_long: 300, r: 0.10818, apr: 0.10448, effectiveApr: 0.09352, profit: 0.1049
i: 0, p_short: 0, p_long: 400, r: 0.10818, apr: 0.11104, effectiveApr: 0.09352, profit: 0.15778
i: 0, p_short: 0, p_long: 500, r: 0.10818, apr: 0.11904, effectiveApr: 0.09352, profit: 0.21441
i: 0, p_short: 100, p_long: 0, r: 0.10818, apr: 0.09277, effectiveApr: 0.09352, profit: -0.00805
i: 0, p_short: 100, p_long: 100, r: 0.10818, apr: 0.09497, effectiveApr: 0.09352, profit: 0.01529
i: 0, p_short: 100, p_long: 200, r: 0.10818, apr: 0.09863, effectiveApr: 0.09352, profit: 0.05178
i: 0, p_short: 100, p_long: 300, r: 0.10818, apr: 0.10374, effectiveApr: 0.09352, profit: 0.09849
i: 0, p_short: 100, p_long: 400, r: 0.10

i: 2, p_short: 200, p_long: 100, r: 0.12578, apr: 0.08864, effectiveApr: 0.08819, profit: 0.0051
i: 2, p_short: 200, p_long: 200, r: 0.12578, apr: 0.09496, effectiveApr: 0.08819, profit: 0.07137
i: 2, p_short: 200, p_long: 300, r: 0.12578, apr: 0.10298, effectiveApr: 0.08819, profit: 0.14365
i: 2, p_short: 200, p_long: 400, r: 0.12578, apr: 0.11268, effectiveApr: 0.08819, profit: 0.21737
i: 2, p_short: 200, p_long: 500, r: 0.12578, apr: 0.12405, effectiveApr: 0.08819, profit: 0.28913
i: 2, p_short: 300, p_long: 0, r: 0.12578, apr: 0.08445, effectiveApr: 0.08819, profit: -0.0443
i: 2, p_short: 300, p_long: 100, r: 0.12578, apr: 0.08908, effectiveApr: 0.08819, profit: 0.01
i: 2, p_short: 300, p_long: 200, r: 0.12578, apr: 0.0954, effectiveApr: 0.08819, profit: 0.07564
i: 2, p_short: 300, p_long: 300, r: 0.12578, apr: 0.10342, effectiveApr: 0.08819, profit: 0.14727
i: 2, p_short: 300, p_long: 400, r: 0.12578, apr: 0.11312, effectiveApr: 0.08819, profit: 0.22038
i: 2, p_short: 300, p_long:

i: 4, p_short: 400, p_long: 200, r: 0.08176, apr: 0.07439, effectiveApr: 0.06653, profit: 0.10565
i: 4, p_short: 400, p_long: 300, r: 0.08176, apr: 0.07864, effectiveApr: 0.06653, profit: 0.15399
i: 4, p_short: 400, p_long: 400, r: 0.08176, apr: 0.08399, effectiveApr: 0.06653, profit: 0.2078
i: 4, p_short: 400, p_long: 500, r: 0.08176, apr: 0.09042, effectiveApr: 0.06653, profit: 0.26416
i: 4, p_short: 500, p_long: 0, r: 0.08176, apr: 0.07254, effectiveApr: 0.06653, profit: 0.08284
i: 4, p_short: 500, p_long: 100, r: 0.08176, apr: 0.07461, effectiveApr: 0.06653, profit: 0.10826
i: 4, p_short: 500, p_long: 200, r: 0.08176, apr: 0.07777, effectiveApr: 0.06653, profit: 0.14448
i: 4, p_short: 500, p_long: 300, r: 0.08176, apr: 0.08202, effectiveApr: 0.06653, profit: 0.18881
i: 4, p_short: 500, p_long: 400, r: 0.08176, apr: 0.08736, effectiveApr: 0.06653, profit: 0.2384
i: 4, p_short: 500, p_long: 500, r: 0.08176, apr: 0.09379, effectiveApr: 0.06653, profit: 0.29061


In [93]:
for result in results:
    fig = px.scatter_3d(result, x='longPool', y='shortPool', z='apr',
                  color='profitable')
    fig.show()

In [117]:
meanProfits = []
for result in results:
    meanProfits.append(result["profit"].mean())
print("Overall mean profit is: %s percent" % (np.mean(meanProfits)*100))

Overall mean profit is: 11.460312886189403 percent
